In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [2]:
# Create a Kalman filter object
kalman = cv2.KalmanFilter(4, 2)

# State [x, y, delta_x, delta_y] where x, y are the position and delta_x, delta_y are the velocity
kalman.statePre = np.zeros((4, 1), dtype=np.float32)

# Transition matrix
kalman.transitionMatrix = np.array([[1, 0, 1, 0],
                                    [0, 1, 0, 1],
                                    [0, 0, 1, 0],
                                    [0, 0, 0, 1]], dtype=np.float32)

# Measurement matrix
kalman.measurementMatrix = np.array([[1, 0, 0, 0],
                                     [0, 1, 0, 0]], dtype=np.float32)

# Process noise covariance matrix
kalman.processNoiseCov = np.array([[1e-4, 0, 0, 0],
                                   [0, 1e-4, 0, 0],
                                   [0, 0, 1e-4, 0],
                                   [0, 0, 0, 1e-4]], dtype=np.float32)

# Measurement noise covariance matrix
kalman.measurementNoiseCov = np.array([[1, 0],
                                       [0, 1]], dtype=np.float32)

# Error covariance matrix
kalman.errorCovPost = np.eye(4, dtype=np.float32)

In [3]:
# Load the video (replace with the path to your video file)
cap = cv2.VideoCapture('test_video.mp4')

# Read the first frame
ret, frame = cap.read()

# Define a region of interest (ROI) to track the object
# For demonstration purposes, let's manually define an ROI.
x, y, w, h = 300, 200, 50, 50  # You can change this to the object position
track_window = (x, y, w, h)

# Initialize the Kalman filter with the first position
kalman.statePre = np.array([[x], [y], [0], [0]], dtype=np.float32)

In [7]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Get the prediction from Kalman filter
    prediction = kalman.predict()
    pred_x, pred_y = int(prediction[0]), int(prediction[1])

    # Draw the predicted position
    cv2.circle(frame, (pred_x, pred_y), 10, (0, 0, 255), -1)

    # Crop the region of interest from the frame (simulating object detection)
    # In a real-world scenario, you would use a tracker like SORT or another object detection model
    roi = frame[y:y+h, x:x+w]
    
    # Update the Kalman filter with the actual position (measured)
    kalman.correct(np.array([[x + w//2], [y + h//2]], dtype=np.float32))

    # Display the tracking window
    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Show the frame
    cv2.imshow('Object Tracking', frame)

    # Wait for the user to press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1295: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'
